In [ ]:
#!pip install scikit-surprise

In [ ]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
import numpy as np

# Data Preparation

## Data Loading

In [ ]:
user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating.columns = ['userID', 'ISBN', 'bookRating']
books = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

b'Skipping line 6451: expected 8 fields, saw 9\nSkipping line 43666: expected 8 fields, saw 10\nSkipping line 51750: expected 8 fields, saw 9\n'
b'Skipping line 92037: expected 8 fields, saw 9\nSkipping line 104318: expected 8 fields, saw 9\nSkipping line 121767: expected 8 fields, saw 9\n'
b'Skipping line 144057: expected 8 fields, saw 9\nSkipping line 150788: expected 8 fields, saw 9\nSkipping line 157127: expected 8 fields, saw 9\nSkipping line 180188: expected 8 fields, saw 9\nSkipping line 185737: expected 8 fields, saw 9\n'
b'Skipping line 209387: expected 8 fields, saw 9\nSkipping line 220625: expected 8 fields, saw 9\nSkipping line 227932: expected 8 fields, saw 11\nSkipping line 228956: expected 8 fields, saw 10\nSkipping line 245932: expected 8 fields, saw 9\nSkipping line 251295: expected 8 fields, saw 9\nSkipping line 259940: expected 8 fields, saw 9\nSkipping line 261528: expected 8 fields, saw 9\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:27

In [ ]:
user.head()

,userID,Location,Age
0,2,"stockton, california, usa",18
1,3,"moscow, yukon territory, russia",\N
2,4,"porto, v.n.gaia, portugal",17
3,5,"farnborough, hants, united kingdom",\N
4,6,"santa monica, california, usa",61


In [ ]:
rating.head()

,userID,ISBN,bookRating
0,276726,0155061224,5
1,276727,0446520802,0
2,276729,052165615X,3
3,276729,0521795028,6
4,276733,2080674722,0


In [ ]:
counts1 = rating['userID'].value_counts()
rating = rating[rating['userID'].isin(counts1[counts1 >= 200].index)]
counts = rating['bookRating'].value_counts()
rating = rating[rating['bookRating'].isin(counts[counts >= 100].index)]

In [ ]:
rating.shape

(527556, 3)

In [ ]:
books.head(3)

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...


In [ ]:
df = pd.merge(user, rating, on='userID', how='inner')
df = pd.merge(df, books, on='ISBN', how='inner')
df_bookTitle = df[['userID','ISBN','bookRating','bookTitle','Location']]
df = df_bookTitle.copy()#.drop('bookTitle',axis='columns')
df 

,userID,ISBN,bookRating,bookTitle,Location
0,254,006000438X,0,The Death of Vishnu: A Novel,"minneapolis, minnesota, usa"
1,4017,006000438X,10,The Death of Vishnu: A Novel,"new orleans, louisiana, usa"
2,6242,006000438X,6,The Death of Vishnu: A Novel,"calgary, alberta, canada"
3,6575,006000438X,9,The Death of Vishnu: A Novel,"seattle, washington, usa"
4,11676,006000438X,6,The Death of Vishnu: A Novel,"n/a, n/a, n/a"
...,...,...,...,...,...
488737,278418,1882419960,0,McSe Systems Management Server 1.2: Rapid Revi...,"omaha, nebraska, usa"
488738,278418,1883672236,0,Fairies: Celebrations from Season to Season (F...,"omaha, nebraska, usa"
488739,278418,1884907989,0,Waters of Eden,"omaha, nebraska, usa"
488740,278418,1892083043,0,Gotcha! Strategy Games for Math and Logic (Hom...,"omaha, nebraska, usa"


In [ ]:
ref_book_title = df_bookTitle[['ISBN','bookTitle']].drop_duplicates()
isbn_list = []
for a in ref_book_title.iterrows():
    #print(a[1][0])
    isbn_list.append([a[1][0],a[1][1]])
isbn_dict = dict(isbn_list)
isbn_dict

{'006000438X': 'The Death of Vishnu: A Novel',
 '0060013117': 'Night Watch',
 '0060199563': 'Thief of Time',
 '0060391448': 'Wicked : The Life and Times of the Wicked Witch of the West',
 '0060502320': "I've Got You, Babe",
 '006052779X': "Charlotte's Web Book and Charm (Charming Classics)",
 '0060563079': 'Peter Pan: The Original Story (Peter Pan)',
 '0060809027': 'My Friend Flicka',
 '0060930535': 'The Poisonwood Bible: A Novel',
 '0060934700': 'Smoke and Mirrors: Short Fictions and Illusions',
 '0060934719': 'stardust',
 '0060976977': 'Amazing Grace : Lives of Children and the Conscience of a Nation, The',
 '0061020699': 'Equal Rites (Discworld Novels (Paperback))',
 '0061020702': 'The Light Fantastic (Discworld Novels (Paperback))',
 '0061020710': 'The Color of Magic',
 '0061040967': 'The Last Hero : A Discworld Fable (Discworld Novels (Hardcover))',
 '0061050474': 'Jingo: A Discworld Novel (Discworld Series/Terry Pratchett)',
 '0061052515': 'Maskerade: A Novel of Discworld (Pratch

In [ ]:
len(isbn_dict)

178371

In [ ]:
df.head()

,userID,ISBN,bookRating,bookTitle,Location
0,254,006000438X,0,The Death of Vishnu: A Novel,"minneapolis, minnesota, usa"
1,4017,006000438X,10,The Death of Vishnu: A Novel,"new orleans, louisiana, usa"
2,6242,006000438X,6,The Death of Vishnu: A Novel,"calgary, alberta, canada"
3,6575,006000438X,9,The Death of Vishnu: A Novel,"seattle, washington, usa"
4,11676,006000438X,6,The Death of Vishnu: A Novel,"n/a, n/a, n/a"


In [ ]:
df.shape

(488742, 5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488742 entries, 0 to 488741
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   userID      488742 non-null  int64 
 1   ISBN        488742 non-null  object
 2   bookRating  488742 non-null  int64 
 3   bookTitle   488742 non-null  object
 4   Location    488742 non-null  object
dtypes: int64(2), object(3)
memory usage: 22.4+ MB


In [ ]:
print('Dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::200000, :])

Dataset shape: (488742, 5)
-Dataset examples-
        userID  ...                     Location
0          254  ...  minneapolis, minnesota, usa
200000  227520  ...    san jose, california, usa
400000  278418  ...         omaha, nebraska, usa

[3 rows x 5 columns]


## Data Filtering

Take only users who rate more than 50 books & books which rated by more than 50 users

In [ ]:
df.head(3)

,userID,ISBN,bookRating,bookTitle,Location
0,254,006000438X,0,The Death of Vishnu: A Novel,"minneapolis, minnesota, usa"
1,4017,006000438X,10,The Death of Vishnu: A Novel,"new orleans, louisiana, usa"
2,6242,006000438X,6,The Death of Vishnu: A Novel,"calgary, alberta, canada"


In [ ]:
df['rating_by_user'] = df.groupby('userID')['bookRating'].transform('count')
df['rating_by_book'] = df.groupby('bookTitle')['bookRating'].transform('count')

In [ ]:
df.shape

(488742, 7)

In [ ]:
df = df[(df['rating_by_book'] >= 50 )] 

In [ ]:
df.head(3)

,userID,ISBN,bookRating,bookTitle,Location,rating_by_user,rating_by_book
68,254,0060930535,0,The Poisonwood Bible: A Novel,"minneapolis, minnesota, usa",300,133
69,6251,0060930535,0,The Poisonwood Bible: A Novel,"wahiawa, hawaii, usa",867,133
70,7346,0060930535,0,The Poisonwood Bible: A Novel,"sunnyvale, california, usa",981,133


In [ ]:
df.shape

(62149, 7)

Take only bookRating > 0 

In [ ]:
#df = df[df['bookRating'] > 0]

In [ ]:
def show_data_profile(df):
    Data_dict = pd.DataFrame(df.dtypes)
    Data_dict = Data_dict.rename(columns={0: 'Data_Type'})
    Data_dict['Missing_Value'] = df.isnull().sum()
    Data_dict['Missing_Val_Pct'] = round((df.isnull().sum()/df.count())*100, 2)
    Data_dict['Unique_Value'] = df.nunique()
    Data_dict['Count'] = df.count()
    return Data_dict.sort_values(by=['Missing_Val_Pct', 'Unique_Value'], ascending=[False, True])

In [ ]:
show_data_profile(df)

,Data_Type,Missing_Value,Missing_Val_Pct,Unique_Value,Count
bookRating,int64,0,0.0,11,62149
rating_by_book,int64,0,0.0,132,62149
rating_by_user,int64,0,0.0,512,62149
Location,object,0,0.0,676,62149
bookTitle,object,0,0.0,746,62149
userID,int64,0,0.0,894,62149
ISBN,object,0,0.0,2259,62149


In [ ]:
df_new = df[df['Location'].str.contains("usa|canada")]
df_new = df_new[['userID','ISBN','bookRating']]

In [ ]:
df_new.describe()

,userID,bookRating
count,56396.000000,56396.000000
mean,142936.726434,1.901926
std,79797.774960,3.503119
min,254.000000,0.000000
25%,76352.000000,0.000000
50%,143253.000000,0.000000
75%,214786.000000,0.000000
max,278418.000000,10.000000


In [ ]:
df_new.shape

(56396, 3)

# Surprise

To load a dataset from a pandas dataframe, we will use the load_from_df() method, we will also need a Reader object, and the rating_scale parameter must be specified. The dataframe must have three columns, corresponding to the user ids, the item ids, and the ratings in this order. Each row thus corresponds to a given rating.

## Data Load

In [ ]:
df_new.head(3)

,userID,ISBN,bookRating
68,254,0060930535,0
69,6251,0060930535,0
70,7346,0060930535,0


In [ ]:
df_new.shape

(56396, 3)

In [ ]:
df_new['bookRating'].value_counts()

0     42932
8      3213
10     3070
9      2735
7      1975
5      1281
6       810
4       177
3        94
1        56
2        53
Name: bookRating, dtype: int64

In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_new[['userID', 'ISBN', 'bookRating']], reader)

In [ ]:
data.df

,userID,ISBN,bookRating
68,254,0060930535,0
69,6251,0060930535,0
70,7346,0060930535,0
72,13273,0060930535,9
73,16634,0060930535,0
...,...,...,...
486905,274808,0449701913,0
487220,275970,1586210661,9
487378,277639,0375402675,0
487418,277639,0590313088,0


With the Surprise library, we will benchmark the following algorithms

### **Basic algorithms**

#### NormalPredictor

* NormalPredictor algorithm predicts a random rating based on the distribution of the training set, which is assumed to be normal. This is one of the most basic algorithms that do not do much work.

#### BaselineOnly

* BasiclineOnly algorithm predicts the baseline estimate for given user and item.

### **k-NN algorithms**

#### KNNBasic

* KNNBasic is a basic collaborative filtering algorithm.

#### KNNWithMeans

* KNNWithMeans is basic collaborative filtering algorithm, taking into account the mean ratings of each user.

#### KNNWithZScore

* KNNWithZScore is a basic collaborative filtering algorithm, taking into account the z-score normalization of each user.

#### KNNBaseline

* KNNBaseline is a basic collaborative filtering algorithm taking into account a baseline rating.

### **Matrix Factorization-based algorithms**

#### SVD

* SVD algorithm is equivalent to Probabilistic Matrix Factorization (http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf)

#### SVDpp

* The SVDpp algorithm is an extension of SVD that takes into account implicit ratings.

#### NMF

* NMF is a collaborative filtering algorithm based on Non-negative Matrix Factorization. It is very similar with SVD.

### Slope One

* Slope One is a straightforward implementation of the SlopeOne algorithm. (https://arxiv.org/abs/cs/0702144)

### Co-clustering

* Co-clustering is a collaborative filtering algorithm based on co-clustering (http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.113.6458&rep=rep1&type=pdf)


We use rmse as our accuracy metric for the predictions.

### Cross Validation

In [ ]:
# results = cross_validate(SVD(), data, measures=['RMSE'], cv=3, verbose=False)
# results

In [ ]:
# benchmark = []
# # Iterate over all algorithms
# for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
#     # Perform cross validation
#     results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
#     # Get results & append algorithm name
#     tmp = pd.DataFrame.from_dict(results).mean(axis=0)
#     tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
#     benchmark.append(tmp)

In [ ]:
# surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [ ]:
# surprise_results

### Build Model

##### Train Test Split

In [ ]:
#Train test split
trainset, testset = train_test_split(data, test_size=0.25)

###### Trainset


In [ ]:
trainset.n_ratings # total num of rows

42297

In [ ]:
trainset.n_items # total uniq item

2024

In [ ]:
trainset.n_users # total uniq user

734

Trainset was converted from pandas dataframe to surprise dataframe

In [ ]:
list_trainset=[]
for row in trainset.all_ratings():
    list_trainset.append(row)
df_trainset = pd.DataFrame(list_trainset,columns=['uid','iid','ratings'])
df_trainset.head(3)

,uid,iid,ratings
0,0,0,0.0
1,0,113,9.0
2,0,190,0.0


To get original ISBN from surprise df, we can use to_raw_iid method

In [ ]:
# print(trainset.to_raw_iid(0))
# print(trainset.to_raw_iid(1531))

In [ ]:
# print(isbn_dict['087113795X'])
# print(isbn_dict['039914496X'])

In [ ]:
list_uid_title=[]
for i in range(0,trainset.n_items):
  try:
    raw_id = trainset.to_raw_iid(i)
    title = isbn_dict[raw_id]
    list_uid_title.append([i,title])
  except:
    list_uid_title.append([i,"<not found>"])

uid_title_dict = dict(list_uid_title)
uid_title_dict

{0: "Bridget Jones's Diary",
 1: 'The Little Friend',
 2: 'Rose Madder',
 3: 'At Home in Mitford (The Mitford Years)',
 4: 'The Bad Beginning (A Series of Unfortunate Events, Book 1)',
 5: 'How Stella Got Her Groove Back',
 6: 'I Capture the Castle',
 7: 'Big Trouble',
 8: 'From the Corner of His Eye',
 9: 'The Summons',
 10: 'Good in Bed',
 11: 'Guilty as Sin',
 12: 'Fahrenheit 451',
 13: 'Point of Origin',
 14: 'The Reptile Room (A Series of Unfortunate Events, Book 2)',
 15: 'The Firm',
 16: 'Evening Class',
 17: 'The Dark Half',
 18: "Everything's Eventual : 14 Dark Tales",
 19: 'Dreamcatcher',
 20: 'The Mummy or Ramses the Damned',
 21: "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 22: 'Love in the Time of Cholera (Penguin Great Books of the 20th Century)',
 23: 'Engaging The Enemy',
 24: 'A Year in Provence',
 25: 'The Tale of the Body Thief (Vampire Chronicles (Paperback))',
 26: 'Wild Animus',
 27: 'The Boy Next Door',
 28: 'Tell No One',
 29: 'All Around

###### Testset

In [ ]:
len(testset)

14099

In [ ]:
testset[0:5]

[(191187, '0440226430', 0.0),
 (112026, '0060928336', 0.0),
 (93085, '0452262143', 0.0),
 (128835, '0425118703', 0.0),
 (127429, '0375401601', 0.0)]

#### BaselineOnly

BaselineOnly algorithm gave us the best rmse, therefore, we will proceed further with BaselineOnly and use Alternating Least Squares (ALS).

In [ ]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'fit_time': (0.05145668983459473, 0.05778932571411133, 0.060720205307006836),
 'test_rmse': array([3.25901027, 3.22428829, 3.23051633]),
 'test_time': (0.10880064964294434, 0.11393857002258301, 0.2521023750305176)}

We use the train_test_split() to sample a trainset and a testset with given sizes, and use the accuracy metric of rmse. We’ll then use the fit() method which will train the algorithm on the trainset, and the test() method which will return the predictions made from the testset

##### Model Fitting

In [ ]:
#Model Fitting
algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


##### Prediction

In [ ]:
# Prediction
predictions = algo.test(testset)
predictions[0:5]

[Prediction(uid=191187, iid='0440226430', r_ui=0.0, est=1, details={'was_impossible': False}),
 Prediction(uid=112026, iid='0060928336', r_ui=0.0, est=1.0098985068919648, details={'was_impossible': False}),
 Prediction(uid=93085, iid='0452262143', r_ui=0.0, est=2.9947481121567776, details={'was_impossible': False}),
 Prediction(uid=128835, iid='0425118703', r_ui=0.0, est=1.6614052480341563, details={'was_impossible': False}),
 Prediction(uid=127429, iid='0375401601', r_ui=0.0, est=1, details={'was_impossible': False})]

##### Evaluation

In [ ]:
#Evaluation
accuracy.rmse(predictions)

RMSE: 3.2397


3.239697768508499

Save model

In [ ]:
#Save and load model
from surprise import dump
dump.dump('surprise_model', predictions, algo)
predictions, algo = dump.load('surprise_model')

In [ ]:
trainset = algo.trainset
print(algo.__class__.__name__)

BaselineOnly


To inspect our predictions in details, we are going to build a pandas data frame with all the predictions.

In [ ]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df_inspect = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_inspect['Iu'] = df_inspect.uid.apply(get_Iu)
df_inspect['Ui'] = df_inspect.iid.apply(get_Ui)
df_inspect['err'] = abs(df_inspect.est - df_inspect.rui)

In [ ]:
df_inspect.head()

,uid,iid,rui,est,details,Iu,Ui,err
0,191187,0440226430,0.0,1.000000,{'was_impossible': False},29,114,1.000000
1,112026,0060928336,0.0,1.009899,{'was_impossible': False},100,156,1.009899
2,93085,0452262143,0.0,2.994748,{'was_impossible': False},41,19,2.994748
3,128835,0425118703,0.0,1.661405,{'was_impossible': False},90,39,1.661405
4,127429,0375401601,0.0,1.000000,{'was_impossible': False},93,12,1.000000


In [ ]:
best_predictions = df_inspect.sort_values(by='err')[:10]
worst_predictions = df_inspect.sort_values(by='err')[-10:]

In [ ]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
13893,135458,0441569595,1.0,1.000000,{'was_impossible': False},51,30,0.000000
5290,233911,0345298349,1.0,1.000000,{'was_impossible': False},19,29,0.000000
1802,233911,0345427637,1.0,1.000000,{'was_impossible': False},19,53,0.000000
13614,136139,0971880107,2.0,1.998451,{'was_impossible': False},28,213,0.001549
1216,55490,034538475X,7.0,6.988614,{'was_impossible': False},34,61,0.011386
10869,123883,0671032992,5.0,5.013176,{'was_impossible': False},45,10,0.013176
13115,114368,0385316879,5.0,5.033706,{'was_impossible': False},75,5,0.033706
13413,208671,0525244581,3.0,3.034714,{'was_impossible': False},66,7,0.034714
13382,114368,0440200563,5.0,5.042183,{'was_impossible': False},75,39,0.042183
10756,114368,0312971125,5.0,4.957374,{'was_impossible': False},75,38,0.042626


The above are the best predictions, and they are not lucky guesses. Because Ui is anywhere between 26 to 146, they are not really small, meaning that significant number of users have rated the target book.

In [ ]:
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
8687,56856,044022165X,10.0,1.0,{'was_impossible': False},132,105,9.0
2947,200674,0446364193,10.0,1.0,{'was_impossible': False},101,94,9.0
4863,238120,080410526X,10.0,1.0,{'was_impossible': False},232,84,9.0
9621,108285,0345413873,10.0,1.0,{'was_impossible': False},39,33,9.0
12968,82893,0671027360,10.0,1.0,{'was_impossible': False},35,110,9.0
8634,230522,0553213172,10.0,1.0,{'was_impossible': False},248,31,9.0
8491,108285,044022165X,10.0,1.0,{'was_impossible': False},39,105,9.0
7935,217375,0375706771,10.0,1.0,{'was_impossible': False},156,100,9.0
8312,227447,0743504771,10.0,1.0,{'was_impossible': False},192,2,9.0
9971,238120,0385413041,10.0,1.0,{'was_impossible': False},232,14,9.0


The worst predictions look pretty surprise. Let's look in more details of the last one ISBN "055358264X", the book was rated by 47 users, user "26544" rated 10, our BaselineOnly algorithm predicts 0.

##### Predicct Single Unseen uid,iid

In [ ]:
uid = 226545  
iid = '0156028778' 

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)
pred

user: 226545     item: 0156028778 r_ui = None   est = 1.29   {'was_impossible': False}


Prediction(uid='226545', iid='0156028778', r_ui=None, est=1.2881450572007114, details={'was_impossible': False})

#### KNN (similarity measure)

##### Model Fitting

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.6/dist-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


##### Compute baseline of users and items

In [ ]:
algo.compute_baselines()

Estimating biases using als...


(array([ 6.46142115e-01,  6.72327641e-01,  1.74943348e-01,  5.60504946e-02,
         1.80120060e-02,  1.70774429e-01, -3.72364332e-01, -1.17753922e-01,
        -6.06989295e-02, -1.67259583e+00, -4.48605566e-01, -9.41871367e-02,
         8.78080184e-01, -4.65659924e-02, -1.40845527e-01, -6.69547235e-02,
         8.66817264e-01, -3.74578695e-01, -1.81666219e-01, -7.74486399e-01,
        -2.38511735e-01,  6.39415000e-01,  3.14514698e-03, -6.06196414e-01,
         2.23208899e-01,  4.32883247e-02, -4.97500796e-01,  8.83806088e-02,
        -5.36277470e-01,  1.56844612e-01,  1.08831805e+00,  4.92077139e-01,
        -5.73234137e-02, -3.67808406e-01,  7.41907701e-01,  2.77456347e-01,
         5.92254339e-02,  1.70895224e-01,  1.94276988e-02, -2.39381000e-01,
        -3.64896497e-01,  3.59709448e-01,  7.24766243e-01, -3.14034444e-01,
        -2.37944496e+00, -1.16326834e-01, -2.02944906e-01, -4.74026153e-01,
        -4.45809681e-01,  3.52824168e-01, -6.95388180e-01, -2.66227026e-01,
        -6.1

##### Compute items similarities

In [ ]:
sim = algo.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
df_sim = pd.DataFrame(sim)
df_sim

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.927333,0.0,1.000000,0.984784,0.0,0.0,1.0,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.000000,0.0,0.984863,0.000000,1.0,0.0,1.0,0.956772,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
df_sim.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532
count,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,...,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000
mean,0.161438,0.012394,0.105620,0.018265,0.186281,0.236624,0.308649,0.270533,0.166669,0.157567,0.059964,0.152425,0.358498,0.372624,0.196513,0.088001,0.133762,0.180350,0.075635,0.091244,0.080826,0.202954,0.112221,0.020222,0.135793,0.265281,0.087232,0.219246,0.318910,0.081134,0.186767,0.222984,0.184784,0.037177,0.201902,0.170580,0.171118,0.150376,0.083943,0.259981,...,0.035225,0.018265,0.011089,0.002609,0.018265,0.012394,0.016960,0.139528,0.006523,0.012394,0.037182,0.037834,0.018265,0.039791,0.022179,0.035225,0.016308,0.013699,0.013699,0.013699,0.018265,0.007175,0.007828,0.059361,0.002609,0.008480,0.037834,0.011742,0.005871,0.033920,0.013046,0.024788,0.035225,0.016960,0.013046,0.026093,0.048924,0.018917,0.033920,0.024136
std,0.367613,0.110672,0.307362,0.133951,0.386547,0.424224,0.460271,0.442838,0.372387,0.364073,0.237398,0.359257,0.478208,0.481307,0.396910,0.283284,0.339674,0.384172,0.264439,0.287917,0.272547,0.401684,0.314801,0.140804,0.341973,0.440009,0.282012,0.412483,0.463635,0.272481,0.389304,0.415536,0.387668,0.189245,0.400481,0.375845,0.376169,0.357172,0.277041,0.437751,...,0.184408,0.133951,0.104755,0.051031,0.133951,0.110672,0.129164,0.346514,0.080528,0.110672,0.189269,0.190858,0.133951,0.195532,0.147313,0.184408,0.126698,0.116275,0.116275,0.116275,0.133951,0.084431,0.088157,0.236376,0.051031,0.091726,0.190858,0.107756,0.076421,0.181083,0.113510,0.155529,0.184408,0.129164,0.113510,0.159463,0.215779,0.136277,0.181083,0.153520
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

##### Get 6 neighbors based on given items id

In [ ]:
algo.get_neighbors(2,6)

[0, 5, 6, 7, 12, 13]

In [ ]:
sim_neighbours=[]
for i in df_sim.index:
  n = algo.get_neighbors(i,6)
  #print(n)
  sim_neighbours.append(n)

df_sim_neighbours = pd.DataFrame(sim_neighbours)
df_sim_neighbours

,0,1,2,3,4,5
0,2,7,8,11,14,16
1,50,114,161,172,398,548
2,0,5,6,7,12,13
3,16,52,82,101,113,130
4,5,7,12,14,15,24
...,...,...,...,...,...,...
1528,24,32,35,84,108,109
1529,22,24,76,81,98,108
1530,21,73,103,159,171,199
1531,59,66,68,74,80,109


In [ ]:
df_sim_neighbours.columns = ['rcmd1','rcmd2','rcmd3','rcmd4','rcmd5','rcmd6']
df_sim_neighbours = df_sim_neighbours.rename(index=uid_title_dict)
df_sim_neighbours

,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
The Mists of Avalon,2,7,8,11,14,16
False Memory,50,114,161,172,398,548
A Civil Action,0,5,6,7,12,13
Fatherhood,16,52,82,101,113,130
Bleachers,5,7,12,14,15,24
...,...,...,...,...,...,...
The Long Road Home,24,32,35,84,108,109
The Wedding,22,24,76,81,98,108
Dreamcatcher,21,73,103,159,171,199
Scarlet Feather,59,66,68,74,80,109


In [ ]:
result = df_sim_neighbours.apply(lambda x : x.map(uid_title_dict))
result

,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
The Mists of Avalon,A Civil Action,Stupid White Men ...and Other Sorry Excuses fo...,Ender's Game (Ender Wiggins Saga (Paperback)),The Rapture of Canaan,Eyes of Prey,American Gods
False Memory,The Hobbit : The Enchanting Prelude to The Lor...,One Door Away from Heaven,Final Target,Cruel &amp; Unusual (Kay Scarpetta Mysteries (...,The Testament,Black House
A Civil Action,The Mists of Avalon,A Wrinkle in Time,She's Come Undone (Oprah's Book Club (Paperback)),Stupid White Men ...and Other Sorry Excuses fo...,Two for the Dough,To Kill a Mockingbird
Fatherhood,American Gods,Nerd in Shining Armor,"Loves Music, Loves to Dance","The Two Towers (The Lord of the Rings, Part 2)",Postmortem,Confessions of a Shopaholic (Summer Display Op...
Bleachers,A Wrinkle in Time,Stupid White Men ...and Other Sorry Excuses fo...,Two for the Dough,Eyes of Prey,Paradise,Fine Things
...,...,...,...,...,...,...
The Long Road Home,Fine Things,Dolores Claiborne,The King of Torts,A Case of Need,The Gift,Three To Get Deadly : A Stephanie Plum Novel (...
The Wedding,Portrait in Death,Fine Things,Key of Knowledge (Key Trilogy (Paperback)),Wifey,The Partner,The Gift
Dreamcatcher,I Know Why the Caged Bird Sings,Nightmares &amp; Dreamscapes,Fahrenheit 451,1984,A Time to Kill,A Prayer for Owen Meany
Scarlet Feather,Miss Julia Speaks Her Mind : A Novel,Jurassic Park,Sphere,The Phantom Tollbooth,Wicked: The Life and Times of the Wicked Witch...,Three To Get Deadly : A Stephanie Plum Novel (...


In [ ]:
result.loc['Chromosome 6']

,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
Chromosome 6,A Wrinkle in Time,F Is for Fugitive (Kinsey Millhone Mysteries (...,Congo,Body of Evidence (Kay Scarpetta Mysteries (Pap...,Like Water for Chocolate: A Novel in Monthly I...,Anne Frank: The Diary of a Young Girl
Chromosome 6,Wifey,The Gift,Bridget Jones: The Edge of Reason,River's End,The Lovely Bones: A Novel,That Camden Summer


In [ ]:
result.loc['Harry Potter and the Order of the Phoenix (Book 5)']

rcmd1    She's Come Undone (Oprah's Book Club (Paperback))
rcmd2    Stupid White Men ...and Other Sorry Excuses fo...
rcmd3        Ender's Game (Ender Wiggins Saga (Paperback))
rcmd4                                The Rapture of Canaan
rcmd5                                    Two for the Dough
rcmd6                                         Eyes of Prey
Name: Harry Potter and the Order of the Phoenix (Book 5), dtype: object

In [ ]:
search_value = 'Chromosome 6'
for key, value in uid_title_dict.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if value == search_value:
        print(key)

1300
1331


##### Predict rating based on given uid and iid

In [ ]:
algo.predict(uid=2,iid=3)

Prediction(uid=2, iid=3, r_ui=None, est=7.969300851653792, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
algo.predict(uid=276726 ,iid='0155061224')

Prediction(uid='276726', iid='0155061224', r_ui=None, est=7.969300851653792, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

##### FULL DATA TRAIN

In [ ]:
df_new#[df_new['bookRating']==1]

,userID,ISBN,bookRating
68,254,0060930535,0
69,6251,0060930535,0
70,7346,0060930535,0
72,13273,0060930535,9
73,16634,0060930535,0
...,...,...,...
486905,274808,0449701913,0
487220,275970,1586210661,9
487378,277639,0375402675,0
487418,277639,0590313088,0


In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56396 entries, 68 to 488496
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userID      56396 non-null  int64 
 1   ISBN        56396 non-null  object
 2   bookRating  56396 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.7+ MB


In [ ]:
df_new.describe()

,userID,bookRating
count,56396.000000,56396.000000
mean,142936.726434,1.901926
std,79797.774960,3.503119
min,254.000000,0.000000
25%,76352.000000,0.000000
50%,143253.000000,0.000000
75%,214786.000000,0.000000
max,278418.000000,10.000000


In [ ]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['userID', 'ISBN', 'bookRating']], reader)

In [ ]:
# Retrieve the trainset.
trainset = data.build_full_trainset()

In [ ]:
trainset

In [ ]:
trainset.n_ratings

56396

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.6/dist-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


In [ ]:
uid = 275970  # raw user id (as in the ratings file). They are **integer**!
iid = '1586210661'  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)
pred



user: 275970     item: 1586210661 r_ui = None   est = 9.00   {'actual_k': 3, 'was_impossible': False}


Prediction(uid=275970, iid='1586210661', r_ui=None, est=9.0, details={'actual_k': 3, 'was_impossible': False})

In [ ]:
sim = algo.compute_similarities()
df_sim = pd.DataFrame(sim)
df_sim

Computing the cosine similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.6/dist-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140
0,1.0,NaN,0.000000,0.000000,0.000000,1.0,0.167273,0.096142,0.299111,0.781899,0.000000,0.399071,0.206431,0.457405,NaN,0.379638,NaN,NaN,NaN,0.408248,0.238265,NaN,NaN,0.000000,0.00000,0.094496,0.000000,1.000000,0.314037,0.227470,NaN,0.224707,0.386227,0.632510,0.384143,0.358444,NaN,NaN,0.898888,0.395240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,1.0,1.000000,NaN,NaN,NaN,0.000000,NaN,1.000000,NaN,NaN,NaN,0.747409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,1.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.000000,0.333758,0.482923,0.0,0.332771,0.517773,0.066944,0.445255,0.413589,0.000000,0.119352,0.171145,NaN,0.413837,NaN,NaN,NaN,0.244266,0.175544,NaN,0.434499,0.612236,0.30317,0.309235,0.309935,0.640184,0.644135,0.428105,0.645788,0.467814,0.655543,0.742124,0.424913,0.275241,NaN,NaN,0.460641,0.296901,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,NaN,0.333758,1.000000,0.000000,NaN,0.311773,0.000000,0.400051,0.351763,0.295420,0.698138,0.668965,0.000000,NaN,0.472207,NaN,NaN,NaN,NaN,0.000000,NaN,0.600000,0.000000,0.00000,0.000000,0.000000,NaN,0.237986,0.261337,0.541318,0.000000,0.173969,0.275630,0.550697,NaN,NaN,0.668965,1.000000,0.581238,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,0.0,NaN,0.482923,0.000000,1.000000,NaN,0.000000,0.458021,0.237913,0.000000,0.000000,0.000000,NaN,0.646997,NaN,0.438875,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.382002,NaN,0.241811,1.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.476618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,0.0,0.0,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2137,0.0,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,0.0,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2138,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,

In [ ]:
#df_sim.fillna(0,inplace = True)

In [ ]:
trainset.n_items

2141

In [ ]:
sim_neighbours=[]
for i in df_sim.index:
  n = algo.get_neighbors(i,6)
  #print(n)
  sim_neighbours.append(n)

df_sim_neighbours = pd.DataFrame(sim_neighbours)
df_sim_neighbours

,0,1,2,3,4,5
0,1,5,9,13,11,2
1,0,2,3,4,5,6
2,1,33,32,30,28,27
3,11,12,0,1,2,4
4,72,0,1,2,5,11
...,...,...,...,...,...,...
2136,0,1,2,3,4,5
2137,22,138,0,1,2,3
2138,0,1,2,3,4,5
2139,0,1,2,3,4,5


In [ ]:
list_uid_title=[]
for i in range(0,trainset.n_items):
  try:
    raw_iid = trainset.to_raw_iid(i)
    title = isbn_dict[raw_iid]
    list_uid_title.append([i,title])
  except:
    list_uid_title.append([i,"<not found>"])

uid_title_dict = dict(list_uid_title)

In [ ]:
pd.DataFrame(list_uid_title)

,0,1
0,0,The Poisonwood Bible: A Novel
1,1,The Alchemist: A Fable About Following Your Dream
2,2,Bridget Jones's Diary
3,3,Bridget Jones: The Edge of Reason
4,4,Chocolat
...,...,...
2136,2136,Homecoming
2137,2137,Me Talk Pretty One Day
2138,2138,Rainbow Six
2139,2139,Hideaway


In [ ]:
df_sim_neighbours.columns = ['rcmd1','rcmd2','rcmd3','rcmd4','rcmd5','rcmd6']
df_sim_neighbours = df_sim_neighbours.rename(index=uid_title_dict)
df_sim_neighbours

,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
The Poisonwood Bible: A Novel,1,5,9,13,11,2
The Alchemist: A Fable About Following Your Dream,0,2,3,4,5,6
Bridget Jones's Diary,1,33,32,30,28,27
Bridget Jones: The Edge of Reason,11,12,0,1,2,4
Chocolat,72,0,1,2,5,11
...,...,...,...,...,...,...
Homecoming,0,1,2,3,4,5
Me Talk Pretty One Day,22,138,0,1,2,3
Rainbow Six,0,1,2,3,4,5
Hideaway,0,1,2,3,4,5


In [ ]:
result = df_sim_neighbours.apply(lambda x : x.map(uid_title_dict))
result

,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Matilda,The Catcher in the Rye,The Cider House Rules,The Mists of Avalon,Bridget Jones's Diary
The Alchemist: A Fable About Following Your Dream,The Poisonwood Bible: A Novel,Bridget Jones's Diary,Bridget Jones: The Edge of Reason,Chocolat,Matilda,The Secret Life of Bees
Bridget Jones's Diary,The Alchemist: A Fable About Following Your Dream,Harry Potter and the Order of the Phoenix (Boo...,Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Chamber of Secrets (Book 2),Fatal Cure
Bridget Jones: The Edge of Reason,The Mists of Avalon,The Witching Hour (Lives of the Mayfair Witches),The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Chocolat
Chocolat,Cause of Death,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Matilda,The Mists of Avalon
...,...,...,...,...,...,...
Homecoming,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Bridget Jones: The Edge of Reason,Chocolat,Matilda
Me Talk Pretty One Day,Coraline,Balzac and the Little Chinese Seamstress : A N...,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Bridget Jones: The Edge of Reason
Rainbow Six,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Bridget Jones: The Edge of Reason,Chocolat,Matilda
Hideaway,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Bridget Jones: The Edge of Reason,Chocolat,Matilda


In [ ]:
result.loc['Chromosome 6']

,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
Chromosome 6,From Potter's Field,Midnight in the Garden of Good and Evil,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream,Bridget Jones's Diary,Bridget Jones: The Edge of Reason
Chromosome 6,Bridget Jones's Diary,Bridget Jones: The Edge of Reason,"Girl, Interrupted",Tell Me Your Dreams,The Poisonwood Bible: A Novel,The Alchemist: A Fable About Following Your Dream


In [ ]:
result.loc['Harry Potter and the Chamber of Secrets (Book 2)']


,rcmd1,rcmd2,rcmd3,rcmd4,rcmd5,rcmd6
Harry Potter and the Chamber of Secrets (Book 2),Chocolat,White Oleander : A Novel,"The Amber Spyglass (His Dark Materials, Book 3)","The Golden Compass (His Dark Materials, Book 1)","The Subtle Knife (His Dark Materials, Book 2)",Neverwhere
Harry Potter and the Chamber of Secrets (Book 2),The Alchemist: A Fable About Following Your Dream,The Mists of Avalon,Bridget Jones's Diary,Matilda,The Hours: A Novel,The Secret Life of Bees
Harry Potter and the Chamber of Secrets (Book 2),The Poisonwood Bible: A Novel,Bridget Jones's Diary,The Alchemist: A Fable About Following Your Dream,Bridget Jones: The Edge of Reason,Chocolat,Matilda


#### SVD (Matrix Factorization)

##### Model Fitting

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

In [ ]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['userID', 'ISBN', 'bookRating']], reader)
# Retrieve the trainset.
trainset = data.build_full_trainset()

In [ ]:

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# First train an SVD algorithm on the movielens dataset.
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
rec_items = []
for uid, user_ratings in top_n.items():
    #print(uid, [iid for (iid, _) in user_ratings])
    rec_items.append([uid,[iid for (iid, _) in user_ratings]])

In [ ]:
df_uid_rec_iid = pd.DataFrame(rec_items, columns=['uid','rec_iid'])
df_rec_iid = pd.DataFrame(df_uid_rec_iid['rec_iid'].tolist(), columns=['rec_iid'+ str(i) for i in range(1,11)])
df_rec_uid_iid = pd.concat([df_uid_rec_iid['uid'],df_rec_iid], axis='columns')
df_rec_uid_iid.set_index('uid', inplace=True)
df_rec_uid_iid


,rec_iid1,rec_iid2,rec_iid3,rec_iid4,rec_iid5,rec_iid6,rec_iid7,rec_iid8,rec_iid9,rec_iid10
uid,,,,,,,,,,
254,059035342X,0312195516,0345342968,0842329129,0842329242,0786868716,0451161351,0786881852,0786867647,0312980140
6251,0439064864,0060502258,0425115801,0553296981,0590353403,006440188X,0345350499,0515130389,044022425X,0439136369
7346,0385484518,0312971346,0380710218,0670835382,0380002930,0439136350,0345386132,0312980159,0440217547,0671042858
13273,0553213148,0312971346,0316769487,0060987561,0743418204,0070212570,006016848X,0590433466,0805063889,0446611085
16634,0446532231,0140143505,0312265867,0312983271,0385504209,0060915544,0439064864,0440241073,0380710218,0440202043
...,...,...,...,...,...,...,...,...,...,...
238961,0345339681,0439064872,0345445848,0316781142,0380002930,0385493800,0671001795,0156528207,0316693200,0385505833
36554,1400031354,0446310786,0345342968,0805063897,0399501487,0060934417,0449223604,1573225789,0743418190,0553213105
139467,0553296981,0439136350,0060392452,0380727501,0618002235,0142001430,0380002930,0451525817,0670835382,0590353403


In [ ]:
df_rec_uid_iid.apply(lambda x : x.map(isbn_dict))

,rec_iid1,rec_iid2,rec_iid3,rec_iid4,rec_iid5,rec_iid6,rec_iid7,rec_iid8,rec_iid9,rec_iid10
uid,,,,,,,,,,
254,Harry Potter and the Sorcerer's Stone (Harry P...,The Red Tent (Bestselling Backlist),Fahrenheit 451,Left Behind: A Novel of the Earth's Last Days ...,Nicolae: The Rise of Antichrist (Left Behind N...,The Five People You Meet in Heaven,Cujo,Don't Sweat the Small Stuff and It's All Small...,Lucky Man: A Memoir,Seven Up (A Stephanie Plum Novel)
6251,Harry Potter and the Chamber of Secrets (Book 2),The Divine Secrets of the Ya-Ya Sisterhood: A ...,Lightning,Anne Frank: The Diary of a Young Girl,Harry Potter and the Sorcerer's Stone (Book 1),The Secret Garden,The Mists of Avalon,Carolina Moon,Drums of Autumn,Harry Potter and the Prisoner of Azkaban (Book 3)
7346,"Tuesdays with Morrie: An Old Man, a Young Man,...",High Five (A Stephanie Plum Novel),Postmortem,Four Past Midnight,Watership Down,Harry Potter and the Prisoner of Azkaban (Book 3),Eyes of a Child,Fast Women,Accident,The Girl Who Loved Tom Gordon
13273,Anne of Avonlea (Anne of Green Gables Novels (...,High Five (A Stephanie Plum Novel),The Catcher in the Rye,I Know This Much Is True,In Her Shoes : A Novel,Fried Green Tomatoes at the Whistle Stop Cafe,"Men Are from Mars, Women Are from Venus: A Pra...",The Secret Garden,Nickel and Dimed: On (Not) Getting By in America,Suzanne's Diary for Nicholas
16634,"Dude, Where's My Country?",84 Charing Cross Road,To the Nines: A Stephanie Plum Novel,Full House (Janet Evanovich's Full Series),The Da Vinci Code,The Bean Trees,Harry Potter and the Chamber of Secrets (Book 2),The Summons,Postmortem,Shell Seekers
...,...,...,...,...,...,...,...,...,...,...
238961,The Hobbit : The Enchanting Prelude to The Lor...,Harry Potter and the Chamber of Secrets (Book 2),Big Cherry Holler: A Big Stone Gap Novel (Ball...,The Last Time They Met : A Novel,Watership Down,The Testament,Two for the Dough,The Little Prince,2nd Chance,Skipping Christmas
36554,Tears of the Giraffe (No.1 Ladies Detective Ag...,To Kill a Mockingbird,Fahrenheit 451,Nickel and Dimed: On (Not) Getting By in America,Lord of the Flies,Bel Canto: A Novel,M Is for Malice,The Color of Water: A Black Man's Tribute to H...,In Her Shoes : A Novel,Pride and Prejudice
139467,Anne Frank: The Diary of a Young Girl,Harry Potter and the Prisoner of Azkaban (Book 3),Stupid White Men ...and Other Sorry Excuses fo...,Notes from a Small Island,"The Two Towers (The Lord of the Rings, Part 2)",Year of Wonders,Watership Down,The Secret Garden,Four Past Midnight,Harry Potter and the Sorcerer's Stone (Book 1)
